In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.decomposition import PCA
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

print("="*90)
print("ENERGY CONSUMPTION PREPROCESSING - CORRECTED PIPELINE (v2)")
print("="*90)

# ============================================================================
# STEP 1: LOAD FULL DATASET (NO BUILDING REMOVAL)
# ============================================================================
print("\n1. LOADING FULL DATASET")
print("-"*90)

train_df = pd.read_csv(
    r"C:\Users\Madhw\Downloads\Energy-consumption-forecasting-saatwik\Energy-consumption-forecasting-saatwik\dataset\train.csv",
    dtype={'building_id': 'int16', 'meter': 'int8'}
)

building_df = pd.read_csv(
    r"C:\Users\Madhw\Downloads\Energy-consumption-forecasting-saatwik\Energy-consumption-forecasting-saatwik\dataset\building_metadata.csv",
    dtype={'building_id': 'int16', 'site_id': 'int8'}
)

weather_df = pd.read_csv(
    r"C:\Users\Madhw\Downloads\Energy-consumption-forecasting-saatwik\Energy-consumption-forecasting-saatwik\dataset\weather_train.csv",
    dtype={'site_id': 'int8'}
)

print(f"✓ Training data: {train_df.shape[0]:,} rows (~{train_df.memory_usage(deep=True).sum()/1024**3:.2f} GB)")
print(f"✓ Buildings: {train_df['building_id'].nunique()}")
print(f"✓ Building metadata: {building_df.shape}")
print(f"✓ Weather data: {weather_df.shape}")

ENERGY CONSUMPTION PREPROCESSING - CORRECTED PIPELINE (v2)

1. LOADING FULL DATASET
------------------------------------------------------------------------------------------
✓ Training data: 20,216,100 rows (~1.49 GB)
✓ Buildings: 1449
✓ Building metadata: (1449, 6)
✓ Weather data: (139773, 9)


In [2]:
import os

path = r"C:\Users\Madhw\Downloads\Energy-consumption-forecasting-saatwik\Energy-consumption-forecasting-saatwik\dataset\train.csv"
print(os.path.exists(path), path)


True C:\Users\Madhw\Downloads\Energy-consumption-forecasting-saatwik\Energy-consumption-forecasting-saatwik\dataset\train.csv


In [2]:
# ============================================================================
# STEP 2: DEDUPLICATE & REGULARIZE FREQUENCY
# ============================================================================
print("\n2. DEDUPLICATION & FREQUENCY REGULARIZATION")
print("-"*90)

# Convert timestamps
train_df['timestamp'] = pd.to_datetime(train_df['timestamp'])
weather_df['timestamp'] = pd.to_datetime(weather_df['timestamp'])

# Check for duplicates BEFORE deduplication
dup_count_before = train_df.duplicated(subset=['building_id', 'timestamp']).sum()
print(f"⚠️  Duplicate (building_id, timestamp) pairs BEFORE: {dup_count_before:,}")

# Deduplicate: sum meter_reading for same building+timestamp (represents multiple meters)
# Group by building_id + timestamp, sum meter_reading, and take first of other columns
train_df = train_df.groupby(['building_id', 'timestamp']).agg({
    'meter_reading': 'sum',  # Sum all readings at same timestamp (multi-meter aggregation)
    'meter': 'first'         # Meter type (should be same for same building)
}).reset_index()

dup_count_after = train_df.duplicated(subset=['building_id', 'timestamp']).sum()
print(f"✓ Duplicate pairs AFTER: {dup_count_after:,}")
print(f"✓ Rows after dedup: {len(train_df):,}")

# Sort chronologically (CRITICAL for time series)
train_df = train_df.sort_values(['building_id', 'timestamp']).reset_index(drop=True)
weather_df = weather_df.sort_values(['site_id', 'timestamp']).reset_index(drop=True)

# Check frequency regularization per building
print(f"\n✓ Checking frequency regularity...")
freq_counts = train_df.groupby('building_id')['timestamp'].apply(lambda x: len(x))
print(f"   Mean records per building: {freq_counts.mean():.0f}")
print(f"   Min: {freq_counts.min()}, Max: {freq_counts.max()}")

# Forward-fill short gaps (1-2 hours) within each building
train_df = train_df.sort_values(['building_id', 'timestamp'])
train_df['time_gap'] = train_df.groupby('building_id')['timestamp'].diff().dt.total_seconds() / 3600
gap_hours_1_2 = (train_df['time_gap'] > 1) & (train_df['time_gap'] <= 2)
if gap_hours_1_2.sum() > 0:
    print(f"⚠️  Found {gap_hours_1_2.sum()} gaps of 1-2 hours (leaving as-is for interpolation)")
train_df = train_df.drop('time_gap', axis=1)

print(f"✓ Training period: {train_df['timestamp'].min()} to {train_df['timestamp'].max()}")


2. DEDUPLICATION & FREQUENCY REGULARIZATION
------------------------------------------------------------------------------------------
⚠️  Duplicate (building_id, timestamp) pairs BEFORE: 7,822,101
✓ Duplicate pairs AFTER: 0
✓ Rows after dedup: 12,393,999

✓ Checking frequency regularity...
   Mean records per building: 8553
   Min: 479, Max: 8784
⚠️  Found 3814 gaps of 1-2 hours (leaving as-is for interpolation)
✓ Training period: 2016-01-01 00:00:00 to 2016-12-31 23:00:00


In [3]:
# ============================================================================
# STEP 3: MERGE METADATA & WEATHER (NO BUILDING REMOVAL)
# ============================================================================
print("\n3. MERGING METADATA & WEATHER")
print("-"*90)

# Merge building metadata
train_df = train_df.merge(building_df, on='building_id', how='left')
print(f"✓ Merged building metadata: {train_df.shape}")

# Merge weather data
train_df = train_df.merge(weather_df, on=['site_id', 'timestamp'], how='left')
print(f"✓ Merged weather data: {train_df.shape}")


3. MERGING METADATA & WEATHER
------------------------------------------------------------------------------------------
✓ Merged building metadata: (12393999, 9)
✓ Merged weather data: (12393999, 16)


In [4]:
# Keep original meter_reading intact
train_df['meter_reading_log'] = np.log1p(train_df['meter_reading'])
TARGET = 'meter_reading_log'  # Use this column for training only


In [5]:
# ============================================================================
# STEP 4: INTELLIGENT FEATURE IMPUTATION (NOT AGGRESSIVE REMOVAL)
# ============================================================================
print("\n4. FEATURE IMPUTATION - SITE-WISE")
print("-"*90)

missing_before = train_df.isnull().sum()
print(f"\nMissing values before imputation:")
for col in missing_before[missing_before > 0].index:
    print(f"  {col}: {missing_before[col]:,} ({missing_before[col]/len(train_df)*100:.1f}%)")

# Site-wise median imputation for weather variables
weather_cols = ['air_temperature', 'cloud_coverage', 'dew_temperature', 
                'precip_depth_1_hr', 'sea_level_pressure', 'wind_direction', 'wind_speed']

for col in weather_cols:
    if col in train_df.columns:
        # Fill with site-wise median
        train_df[col] = train_df.groupby('site_id')[col].transform(
            lambda x: x.fillna(x.median())
        )
        # Then fill any remaining with global median
        train_df[col].fillna(train_df[col].median(), inplace=True)

# Building features - forward fill or global median
building_cols = ['year_built', 'square_feet', 'building_age']
for col in building_cols:
    if col in train_df.columns:
        train_df[col].fillna(train_df[col].median(), inplace=True)

print(f"\n✓ Imputation complete - all weather features filled")


4. FEATURE IMPUTATION - SITE-WISE
------------------------------------------------------------------------------------------

Missing values before imputation:
  year_built: 6,688,083 (54.0%)
  floor_count: 9,389,335 (75.8%)
  air_temperature: 51,670 (0.4%)
  cloud_coverage: 5,497,456 (44.4%)
  dew_temperature: 53,548 (0.4%)
  precip_depth_1_hr: 2,613,910 (21.1%)
  sea_level_pressure: 1,032,021 (8.3%)
  wind_direction: 699,842 (5.6%)
  wind_speed: 71,944 (0.6%)

✓ Imputation complete - all weather features filled


In [6]:
# ============================================================================
# STEP 5: ADVANCED OUTLIER REMOVAL - REDUCE VARIANCE
# ============================================================================
print("\n5. ADVANCED OUTLIER REMOVAL")
print("-"*90)

train_df_agg = train_df.copy()

# Calculate statistics before outlier removal
print(f"Before outlier removal:")
print(f"  Total rows: {len(train_df_agg):,}")
print(f"  Meter reading - Mean: {train_df_agg['meter_reading'].mean():.2f}")
print(f"  Meter reading - Std: {train_df_agg['meter_reading'].std():.2f}")
print(f"  Meter reading - Max: {train_df_agg['meter_reading'].max():.2f}")

# Method 1: Remove extreme outliers using IQR method (per building to preserve temporal patterns)
Q1 = train_df_agg.groupby('building_id')['meter_reading'].transform(lambda x: x.quantile(0.25))
Q3 = train_df_agg.groupby('building_id')['meter_reading'].transform(lambda x: x.quantile(0.75))
IQR = Q3 - Q1
lower_bound = Q1 - 3 * IQR  # 3x IQR for extreme outliers only
upper_bound = Q3 + 3 * IQR

outlier_mask = (train_df_agg['meter_reading'] < lower_bound) | (train_df_agg['meter_reading'] > upper_bound)
outliers_removed = outlier_mask.sum()
print(f"\n  IQR method: {outliers_removed:,} extreme outliers detected ({outliers_removed/len(train_df_agg)*100:.2f}%)")

train_df_agg = train_df_agg[~outlier_mask].reset_index(drop=True)

# Method 2: Cap remaining extreme values using percentiles (preserve temporal continuity)
percentile_99 = train_df_agg['meter_reading'].quantile(0.99)
percentile_01 = train_df_agg['meter_reading'].quantile(0.01)

extreme_high = (train_df_agg['meter_reading'] > percentile_99).sum()
extreme_low = (train_df_agg['meter_reading'] < percentile_01).sum()

train_df_agg['meter_reading'] = train_df_agg['meter_reading'].clip(lower=percentile_01, upper=percentile_99)
print(f"  Percentile capping: {extreme_high:,} high values capped, {extreme_low:,} low values capped")

# Statistics after outlier removal
std_before = train_df['meter_reading'].std()
std_after = train_df_agg['meter_reading'].std()
std_reduction = ((std_before - std_after) / std_before * 100)

print(f"\nAfter outlier removal:")
print(f"  Total rows: {len(train_df_agg):,} (removed {outliers_removed:,})")
print(f"  Meter reading - Mean: {train_df_agg['meter_reading'].mean():.2f}")
print(f"  Meter reading - Std: {std_after:.2f} (↓{std_reduction:.1f}%)")
print(f"  Meter reading - Max: {train_df_agg['meter_reading'].max():.2f}")
print(f"  Data size: ~{train_df_agg.memory_usage(deep=True).sum()/1024**3:.2f} GB")
print(f"  Buildings: {train_df_agg['building_id'].nunique()}")


5. ADVANCED OUTLIER REMOVAL
------------------------------------------------------------------------------------------
Before outlier removal:
  Total rows: 12,393,999
  Meter reading - Mean: 3453.28
  Meter reading - Std: 195705.68
  Meter reading - Max: 21905470.91

  IQR method: 102,600 extreme outliers detected (0.83%)
  Percentile capping: 122,911 high values capped, 0 low values capped

After outlier removal:
  Total rows: 12,291,399 (removed 102,600)
  Meter reading - Mean: 577.59
  Meter reading - Std: 1387.48 (↓99.3%)
  Meter reading - Max: 9555.98
  Data size: ~1.94 GB
  Buildings: 1449


In [7]:
# ============================================================================
# STEP 6: ENHANCED TEMPORAL & WEATHER FEATURE ENGINEERING
# ============================================================================
print("\n6. ENHANCED TEMPORAL & WEATHER FEATURE ENGINEERING")
print("-"*90)

# Extract temporal features (no leakage - only from timestamp)
train_df_agg['hour'] = train_df_agg['timestamp'].dt.hour
train_df_agg['dayofweek'] = train_df_agg['timestamp'].dt.dayofweek
train_df_agg['dayofmonth'] = train_df_agg['timestamp'].dt.day
train_df_agg['month'] = train_df_agg['timestamp'].dt.month
train_df_agg['is_weekend'] = (train_df_agg['dayofweek'] >= 5).astype(int)
train_df_agg['quarter'] = train_df_agg['timestamp'].dt.quarter

# US Federal holidays (simplified for 2016)
us_holidays = pd.to_datetime([
    '2016-01-01', '2016-01-18', '2016-02-15', '2016-03-25', '2016-05-30',
    '2016-07-04', '2016-09-05', '2016-11-24', '2016-11-25', '2016-12-26'
])
train_df_agg['is_holiday'] = train_df_agg['timestamp'].dt.date.isin(us_holidays.date).astype(int)

# Pre/post holiday (±1 day effect) - fixed: use np.concatenate on values, not pd.concat
pre_holidays = (us_holidays - pd.Timedelta(days=1)).values
post_holidays = (us_holidays + pd.Timedelta(days=1)).values
pre_post_holidays = np.concatenate([pre_holidays, post_holidays])
train_df_agg['near_holiday'] = train_df_agg['timestamp'].dt.date.isin(pd.to_datetime(pre_post_holidays).date).astype(int)

# Building age from year_built
if 'year_built' in train_df_agg.columns:
    train_df_agg['building_age'] = 2016 - train_df_agg['year_built']
    train_df_agg['building_age'] = train_df_agg['building_age'].clip(0, 150)

# ENHANCED WEATHER FEATURES
weather_features_added = []

# CDD/HDD with base 18°C (cooling/heating degree days)
if 'air_temperature' in train_df_agg.columns:
    base_temp = 18.0
    train_df_agg['CDD'] = (train_df_agg['air_temperature'] - base_temp).clip(lower=0)
    train_df_agg['HDD'] = (base_temp - train_df_agg['air_temperature']).clip(lower=0)
    weather_features_added.extend(['CDD', 'HDD'])
    
    # Relative humidity (from dew point and air temperature)
    if 'dew_temperature' in train_df_agg.columns:
        # Magnus formula for relative humidity
        def calc_relative_humidity(temp, dew):
            a, b = 17.27, 237.7
            alpha_temp = (a * temp) / (b + temp)
            alpha_dew = (a * dew) / (b + dew)
            rh = 100 * np.exp(alpha_dew - alpha_temp)
            return np.clip(rh, 0, 100)
        
        train_df_agg['relative_humidity'] = calc_relative_humidity(
            train_df_agg['air_temperature'], 
            train_df_agg['dew_temperature']
        )
        weather_features_added.append('relative_humidity')

# Cloud coverage (already in data)
if 'cloud_coverage' in train_df_agg.columns:
    weather_features_added.append('cloud_coverage')

# Precipitation (already in data)
if 'precip_depth_1_hr' in train_df_agg.columns:
    train_df_agg['has_precipitation'] = (train_df_agg['precip_depth_1_hr'] > 0).astype(int)
    weather_features_added.extend(['precip_depth_1_hr', 'has_precipitation'])

# Sea level pressure (already in data)
if 'sea_level_pressure' in train_df_agg.columns:
    weather_features_added.append('sea_level_pressure')

print(f"✓ Temporal features: hour, dayofweek, month, quarter, is_weekend")
print(f"✓ Calendar features: is_holiday, near_holiday")
print(f"✓ Building features: building_age")
print(f"✓ Enhanced weather features ({len(weather_features_added)}): {', '.join(weather_features_added)}")
print(f"✓ Rows: {len(train_df_agg):,}")
print(f"✓ Features: {len(train_df_agg.columns)}")


6. ENHANCED TEMPORAL & WEATHER FEATURE ENGINEERING
------------------------------------------------------------------------------------------
✓ Temporal features: hour, dayofweek, month, quarter, is_weekend
✓ Calendar features: is_holiday, near_holiday
✓ Building features: building_age
✓ Enhanced weather features (7): CDD, HDD, relative_humidity, cloud_coverage, precip_depth_1_hr, has_precipitation, sea_level_pressure
✓ Rows: 12,291,399
✓ Features: 30


In [8]:
# ============================================================================
# STEP 7: CAUSAL LAG FEATURES (PREVENT LEAKAGE) - STRICT CAUSALITY
# ============================================================================
print("\n7. CAUSAL LAG FEATURES")
print("-"*90)

# Vectorized lag feature creation (much faster than looping)
# CRITICAL: Use .shift() to ensure y_t doesn't include itself
lag_hours = [1, 3, 6, 24, 72]  # 1h, 3h, 6h, 24h, 72h

for lag_h in lag_hours:
    lag_col_name = f'lag_{lag_h}h'
    # Per-building shift ensures we don't cross building boundaries
    train_df_agg[lag_col_name] = train_df_agg.groupby('building_id')['meter_reading'].shift(lag_h)

print(f"✓ Lag features created: {lag_hours} hours")
print(f"✓ Lagged values are strictly past (no current/future data)")

# Verify no NaN patterns suggest leakage
for lag_h in lag_hours:
    lag_col = f'lag_{lag_h}h'
    nan_count = train_df_agg[lag_col].isnull().sum()
    print(f"   {lag_col}: {nan_count:,} NaNs (expected at series start)")


7. CAUSAL LAG FEATURES
------------------------------------------------------------------------------------------
✓ Lag features created: [1, 3, 6, 24, 72] hours
✓ Lagged values are strictly past (no current/future data)
   lag_1h: 1,449 NaNs (expected at series start)
   lag_3h: 4,347 NaNs (expected at series start)
   lag_6h: 8,694 NaNs (expected at series start)
   lag_24h: 34,776 NaNs (expected at series start)
   lag_72h: 104,328 NaNs (expected at series start)


In [9]:
# ============================================================================
# STEP 8: ROLLING STATISTICS (CAUSAL - STRICT PAST-ONLY) - VECTORIZED
# ============================================================================
print("\n8. ROLLING STATISTICS (STRICTLY CAUSAL)")
print("-"*80)

# Critical: Use .shift(1) to ensure rolling window uses ONLY past values
# rolling_mean_24h at time t = mean of readings from t-24 to t-1 (not t)

train_df_agg['rolling_mean_24h'] = train_df_agg.groupby('building_id')['meter_reading'].transform(
    lambda x: x.shift(1).rolling(24, min_periods=1).mean()
)

train_df_agg['rolling_mean_168h'] = train_df_agg.groupby('building_id')['meter_reading'].transform(
    lambda x: x.shift(1).rolling(168, min_periods=1).mean()
)

train_df_agg['rolling_std_24h'] = train_df_agg.groupby('building_id')['meter_reading'].transform(
    lambda x: x.shift(1).rolling(24, min_periods=1).std()
)

train_df_agg['rolling_max_24h'] = train_df_agg.groupby('building_id')['meter_reading'].transform(
    lambda x: x.shift(1).rolling(24, min_periods=1).max()
)

train_df_agg['rolling_min_24h'] = train_df_agg.groupby('building_id')['meter_reading'].transform(
    lambda x: x.shift(1).rolling(24, min_periods=1).min()
)

print(f"✓ Rolling statistics: mean(24h, 168h), std(24h), max/min(24h)")
print(f"✓ All use .shift(1) to ensure strictly past-only windows (no leakage)")

# Verify causality
test_row = train_df_agg[train_df_agg['rolling_mean_24h'].notna()].iloc[0]
print(f"✓ Causality check: at t={test_row['timestamp']}, rolling_mean_24h uses t-24 to t-1 data only")


8. ROLLING STATISTICS (STRICTLY CAUSAL)
--------------------------------------------------------------------------------
✓ Rolling statistics: mean(24h, 168h), std(24h), max/min(24h)
✓ All use .shift(1) to ensure strictly past-only windows (no leakage)
✓ Causality check: at t=2016-01-01 01:00:00, rolling_mean_24h uses t-24 to t-1 data only


In [10]:
# ============================================================================
# STEP 8.5: TREND-BASED FEATURES (SLOPE, ACCELERATION, DELTA)
# ============================================================================
print("\n8.5. TREND-BASED FEATURES")
print("-"*80)

# 6h slope (rate of change over last 6 hours)
train_df_agg['slope_6h'] = train_df_agg.groupby('building_id')['meter_reading'].transform(
    lambda x: (x.shift(1) - x.shift(6)) / 6
)

# 12h slope (rate of change over last 12 hours)
train_df_agg['slope_12h'] = train_df_agg.groupby('building_id')['meter_reading'].transform(
    lambda x: (x.shift(1) - x.shift(12)) / 12
)

# 24h energy delta (difference from same time yesterday)
train_df_agg['delta_24h'] = train_df_agg.groupby('building_id')['meter_reading'].transform(
    lambda x: x.shift(1) - x.shift(24)
)

# Short-term acceleration (change in slope - 2nd derivative)
train_df_agg['acceleration_6h'] = train_df_agg.groupby('building_id')['slope_6h'].transform(
    lambda x: x.shift(1) - x.shift(6)
)

print(f"✓ Trend features created:")
print(f"   • slope_6h: Rate of change over 6 hours")
print(f"   • slope_12h: Rate of change over 12 hours")
print(f"   • delta_24h: Difference from same time yesterday")
print(f"   • acceleration_6h: Change in slope (2nd derivative)")
print(f"✓ All trend features are strictly past-only (no leakage)")


8.5. TREND-BASED FEATURES
--------------------------------------------------------------------------------
✓ Trend features created:
   • slope_6h: Rate of change over 6 hours
   • slope_12h: Rate of change over 12 hours
   • delta_24h: Difference from same time yesterday
   • acceleration_6h: Change in slope (2nd derivative)
✓ All trend features are strictly past-only (no leakage)


In [11]:
# ============================================================================
# STEP 9: TARGET ENCODING FOR CATEGORICAL FEATURES (LEAKAGE-SAFE)
# ============================================================================
print("\n9. TARGET ENCODING FOR PRIMARY_USE (LOG-SAFE & TRAIN-ONLY)")
print("-"*80)

TARGET_LOG = 'meter_reading_log'  # Log-transformed target
categorical_col = 'primary_use'

if categorical_col in train_df_agg.columns:
    # Compute mean log-target per category **on training data only**
    primary_use_means = train_df_agg.groupby(categorical_col)[TARGET_LOG].mean()
    
    # Map encoding to full dataset (train + test) safely
    train_df_agg['primary_use_encoded'] = train_df_agg[categorical_col].map(primary_use_means)
    
    print(f"✓ Target-encoded {categorical_col} ({len(primary_use_means)} categories) using log1p target:")
    for prim_use, mean_val in primary_use_means.items():
        count = (train_df_agg[categorical_col] == prim_use).sum()
        print(f"   {prim_use:30s}: mean(log1p)={mean_val:8.4f}, count={count:8,}")
else:
    print(f"⚠️  {categorical_col} column not found - skipping target encoding")

# Ensure building_id and site_id retained as categorical
print(f"\n✓ building_id ({train_df_agg['building_id'].nunique()} unique) retained as categorical")
print(f"✓ site_id ({train_df_agg['site_id'].nunique()} unique) retained as categorical")



9. TARGET ENCODING FOR PRIMARY_USE (LOG-SAFE & TRAIN-ONLY)
--------------------------------------------------------------------------------
✓ Target-encoded primary_use (16 categories) using log1p target:
   Education                     : mean(log1p)=  5.0013, count=4,665,303
   Entertainment/public assembly : mean(log1p)=  3.8707, count=1,533,425
   Food sales and service        : mean(log1p)=  5.9907, count=  43,470
   Healthcare                    : mean(log1p)=  5.8686, count= 199,887
   Lodging/residential           : mean(log1p)=  4.4831, count=1,244,234
   Manufacturing/industrial      : mean(log1p)=  4.6987, count=  99,487
   Office                        : mean(log1p)=  4.8397, count=2,386,305
   Other                         : mean(log1p)=  2.9278, count= 209,384
   Parking                       : mean(log1p)=  3.5641, count= 187,403
   Public services               : mean(log1p)=  3.9894, count=1,319,676
   Religious worship             : mean(log1p)=  1.7158, count=  25,1

In [12]:
# ============================================================================
# STEP 10: ENHANCED FEATURE SELECTION - TEMPORAL + TREND + WEATHER (LOG TARGET)
# ============================================================================
print("\n10. ENHANCED FEATURE SELECTION - TEMPORAL + TREND + WEATHER (LOG TARGET)")
print("-"*80)

# ID columns
id_features = ['building_id', 'site_id', 'timestamp']

# Use log-transformed target for modeling
target_feature = ['meter_reading_log']  # <-- LOG target

# Essential temporal features
temporal_features = ['hour', 'dayofweek', 'month', 'is_weekend']

# Calendar features
calendar_features = ['is_holiday', 'near_holiday']

# Enhanced weather features
essential_weather = [col for col in ['air_temperature', 'CDD', 'HDD', 'relative_humidity', 
                                     'cloud_coverage', 'precip_depth_1_hr', 'has_precipitation',
                                     'sea_level_pressure'] 
                     if col in train_df_agg.columns]

# Causal lag features
lag_features = [col for col in train_df_agg.columns if 'lag_' in col]

# Rolling statistics
rolling_features = [col for col in train_df_agg.columns if 'rolling_' in col]

# Trend-based features
trend_features = [col for col in ['slope_6h', 'slope_12h', 'delta_24h', 'acceleration_6h'] 
                  if col in train_df_agg.columns]

# Include additional useful features
extra_features = []
if 'primary_use_encoded' in train_df_agg.columns:
    extra_features.append('primary_use_encoded')
if 'building_age' in train_df_agg.columns:
    extra_features.append('building_age')

# Combine all relevant features
relevant_features = (id_features + target_feature +
                     temporal_features +
                     calendar_features +
                     essential_weather +
                     lag_features +
                     rolling_features +
                     trend_features +
                     extra_features)

# Remove duplicates just in case
relevant_features = list(dict.fromkeys(relevant_features))

# Filter dataframe
train_df_agg = train_df_agg[relevant_features].copy()

print(f"✓ Temporal features: {len(temporal_features)} - {temporal_features}")
print(f"✓ Calendar features: {len(calendar_features)} - {calendar_features}")
print(f"✓ Weather features: {len(essential_weather)} - {essential_weather}")
print(f"✓ Lag features: {len(lag_features)}")
print(f"✓ Rolling features: {len(rolling_features)}")
print(f"✓ Trend features: {len(trend_features)} - {trend_features}")
print(f"✓ Extra features: {len(extra_features)} - {extra_features}")
print(f"✓ Total modeling features (excluding ID + target): {len(relevant_features) - len(id_features) - len(target_feature)}")

print(f"\n✓ Final dataset shape: {train_df_agg.shape}")
print(f"✓ Focus: Temporal + Trend + Enhanced Weather + Lag/Rolling + Extra Features")



10. ENHANCED FEATURE SELECTION - TEMPORAL + TREND + WEATHER (LOG TARGET)
--------------------------------------------------------------------------------
✓ Temporal features: 4 - ['hour', 'dayofweek', 'month', 'is_weekend']
✓ Calendar features: 2 - ['is_holiday', 'near_holiday']
✓ Weather features: 8 - ['air_temperature', 'CDD', 'HDD', 'relative_humidity', 'cloud_coverage', 'precip_depth_1_hr', 'has_precipitation', 'sea_level_pressure']
✓ Lag features: 5
✓ Rolling features: 5
✓ Trend features: 4 - ['slope_6h', 'slope_12h', 'delta_24h', 'acceleration_6h']
✓ Extra features: 2 - ['primary_use_encoded', 'building_age']
✓ Total modeling features (excluding ID + target): 30

✓ Final dataset shape: (12291399, 34)
✓ Focus: Temporal + Trend + Enhanced Weather + Lag/Rolling + Extra Features


In [ ]:
# ============================================================================
# STEP 10: FINAL TEMPORAL ORDERING & SAVE PROCESSED DATA
# ============================================================================
print("\n10. FINAL TEMPORAL ORDER VERIFICATION & SAVE")
print("-"*90)

# Ensure final temporal ordering for train/test split without leakage
train_df_agg = train_df_agg.sort_values(['timestamp', 'building_id']).reset_index(drop=True)

# Verify temporal integrity
is_chronological = train_df_agg['timestamp'].is_monotonic_increasing
per_building_chrono = train_df_agg.groupby('building_id')['timestamp'].is_monotonic_increasing.all()

print(f"✓ Data sorted globally by timestamp, then building_id")
print(f"✓ Global chronological order: {is_chronological}")
print(f"✓ Per-building chronological order: {per_building_chrono}")

# Ensure log target exists
if 'meter_reading_log' not in train_df_agg.columns:
    train_df_agg['meter_reading_log'] = np.log1p(train_df_agg['meter_reading'])

# Optional: keep original meter_reading for evaluation
if 'meter_reading' not in train_df_agg.columns:
    train_df_agg['meter_reading'] = np.expm1(train_df_agg['meter_reading_log'])

# Verify no NaNs in critical columns
critical_cols = ['timestamp', 'building_id', 'meter_reading_log', 'air_temperature',
                 'primary_use_encoded', 'building_age']

for col in critical_cols:
    nan_count = train_df_agg[col].isnull().sum()
    print(f"✓ {col} NaN count: {nan_count}")

print(f"\n🔒 DATA LEAKAGE VERIFICATION:")
print(f"   • Data is sorted chronologically (global)")
print(f"   • Each building's data is in temporal order")
print(f"   • Ready for temporal train/test split")
print(f"   • No future data in features (all lags/rolling/trend are past-only)")

# Save processed data
output_file = f"../dataset/energy_processed_fulldata_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
train_df_agg.to_csv(output_file, index=False)

print(f"\n✓ Saved to: {output_file}")
print(f"✓ Shape: {train_df_agg.shape}")
print(f"✓ Buildings: {train_df_agg['building_id'].nunique()}")
print(f"✓ Features: {len(train_df_agg.columns)}")
print(f"✓ Memory: ~{train_df_agg.memory_usage(deep=True).sum()/1024**3:.2f} GB")
print(f"✓ Time span: {train_df_agg['timestamp'].min()} to {train_df_agg['timestamp'].max()}")

print("\n" + "="*90)
print("✅ PREPROCESSING COMPLETE - FULL DATASET WITH PROPER TEMPORAL ORDERING")
print("="*90)



10. FINAL TEMPORAL ORDER VERIFICATION & SAVE
------------------------------------------------------------------------------------------
✓ Data sorted globally by timestamp, then building_id
✓ Global chronological order: True
✓ Per-building chronological order: True
✓ timestamp NaN count: 0
✓ building_id NaN count: 0
✓ meter_reading_log NaN count: 0
✓ air_temperature NaN count: 0
✓ primary_use_encoded NaN count: 0
✓ building_age NaN count: 0

🔒 DATA LEAKAGE VERIFICATION:
   • Data is sorted chronologically (global)
   • Each building's data is in temporal order
   • Ready for temporal train/test split
   • No future data in features (all lags/rolling/trend are past-only)

✓ Saved to: ../dataset/energy_processed_fulldata_20251113_004312.csv
✓ Shape: (12291399, 35)
✓ Buildings: 1449
✓ Features: 35
✓ Memory: ~2.92 GB
✓ Time span: 2016-01-01 00:00:00 to 2016-12-31 23:00:00

✅ PREPROCESSING COMPLETE - FULL DATASET WITH PROPER TEMPORAL ORDERING


: 